# Sample ROIs
This notebook performs the following:
* fixes the resolution / scale issue between ECM and cell location by doubling the index of cell locations.
* samples regions of size 4,000 by 4,000 from ROIs.
* link to polyscope: https://docs.google.com/spreadsheets/d/1CXmBIhaI2vvPFNOqSaxXDzO1_3G4RQqc/edit#gid=1637067803

In [1]:
include("../src/ECM_TDA.jl")
using .ECM_TDA

using Images
using CSV
using Plots
using JLD2
using Random
using StatsBase


WebIO._IJuliaInit()

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/yJrrq/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010de222cf, Ptr{Nothing} @0x000000010deb7206, Ptr{Nothing} @0x000000010deb823b, Ptr{Nothing} @0x000000010deb6b94, Ptr{Nothing} @0x000000010deb6f0c, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010ded1844, Ptr{Nothing} @0x000000010ded0f65, Ptr{Nothing} @0x000000010ded1604, Ptr{Nothing} @0x000000010ded1604, Ptr{Nothing} @0x000000010ded2134, Ptr{Nothing} @0x000000011f9414e7, Ptr{Nothing} @0x000000010dea0fbf, Ptr{Nothing} @0x000000011f992312, Ptr{Nothing} @0x000000011f5c8a65, Ptr{Nothing} @0x000000010dea0fbf, Ptr{Nothing} @0x000000010dead839, Ptr{Nothing} @0x000000010f3f1bae, Ptr{Nothing} @0x000000010dea0fbf, Ptr{Nothing} @0x000000010deb825f, Ptr{Nothing} @0x000000010deb6b94, Ptr{Nothing} @0x000000

# 1. Load all (LTX, Da)

ROIs in two batches
1. `data/ROI`
    * combined folder from the first three batch of ROIs
2. `ROI_ES_201222` 
    * added in early 2023
    * further split into `green` and `purple`

## 1(a) Load all LTX_Da

In [ ]:
### get LTX_Da from first ROIs
csv_dir = "data/ROI/region_csv/"
csv_files = [item for item in walkdir(csv_dir)]
csv_files = csv_files[2:end]

# organize file structures
LTX_Da = Dict()
for file in csv_files
    dir_name = file[1]
    LTX = split(split(dir_name,".")[1], "/")[end][4:6]
    Da_all = [String(split(item,".")[1][3:end]) for item in file[3]]

    if Da_all != []
        LTX_Da[LTX] = Da_all
    end
end

print("Total number of ROIs: ", length(LTX_Da))

# save 
#save("data/ROI/LTX_Da_dict.jld2", "LTX_Da", LTX_Da)

In [31]:
### get LTX_Da from the second ROIs
csv_dir = "data/ROI_ES_201222/deconvolved/stitch_region_40x_green/"
csv_files = [item for item in walkdir(csv_dir)]
csv_files = csv_files[2:end]

LTX_Da_green = Dict{Any, Any}()
for file in csv_files
    dir_name = file[1]
    LTX = split(split(dir_name,".")[1], "/")[end][4:6]
    Da_all = String.([split(item, "_")[2][3:end] for item in file[3]])

    if Da_all != []
        LTX_Da_green[LTX] = Da_all
    end
end

# save 
#save("data/ROI_ES_201222/deconvolved/LTX_Da_green_dict.jld2", "LTX_Da_green", LTX_Da_green)

In [34]:
### get LTX_Da from the second ROIs
csv_dir = "data/ROI_ES_201222/deconvolved/stitch_region_40x_purple/"
csv_files = [item for item in walkdir(csv_dir)]
csv_files = csv_files[2:end]

LTX_Da_purple = Dict{Any, Any}()
for file in csv_files
    dir_name = file[1]
    LTX = split(split(dir_name,".")[1], "/")[end][4:6]
    Da_all = String.([split(item, "_")[2][3:end] for item in file[3]])

    if Da_all != []
        LTX_Da_purple[LTX] = Da_all
    end
end

# save 
#save("data/ROI_ES_201222/deconvolved/LTX_Da_purple_dict.jld2", "LTX_Da_purple", LTX_Da_purple)

In [35]:
# load all LTX_Da
LTX_Da = load("data/ROI/LTX_Da_dict.jld2")["LTX_Da"]
LTX_Da_green = load("data/ROI_ES_201222/deconvolved/LTX_Da_green_dict.jld2")["LTX_Da_green"]
LTX_Da_purple = load("data/ROI_ES_201222/deconvolved/LTX_Da_purple_dict.jld2")["LTX_Da_purple"];

## 1(b) Check that images have similar scales across the first and second batch 

In [ ]:
# load all LTX_Da
LTX_Da = load("data/ROI/LTX_Da_dict.jld2")["LTX_Da"]
LTX_Da_green = load("data/ROI_ES_201222/deconvolved/LTX_Da_green_dict.jld2")["LTX_Da_green"]
LTX_Da_purple = load("data/ROI_ES_201222/deconvolved/LTX_Da_purple_dict.jld2")["LTX_Da_purple"];

In [41]:
# check if there are overlapping keys
println("intersecting ROIs: ")
print(intersect(keys(LTX_Da), keys(LTX_Da_green), keys(LTX_Da_purple)))

intersecting ROIs: 
Set(Any["210", "021", "100", "051", "097", "049", "050"])

In [62]:
# Box 62
img = load_ECM_image("021", "110")
println("size: ", size(img))

size: (12000, 8000)


In [66]:
# top green 
img_green = load_ECM_image("021", "106"; green_purple = "green")
println("size: ", size(img_green))

size: (8000, 8000)


# 2. Save cell location after doubling the index

* For images in `data/ROI`: For each CSV file in `region_csv`, double the index location and save in `region_csv_scale_corrected`.

In [34]:
for (LTX, Da_list) in LTX_Da
    for Da in Da_list
        old_path = "data/ROI/region_csv/LTX" * LTX * "PSR_20X.czi/Da" * Da * ".csv"
        
        # load cell locations, double the index
        cells = CSV.read(old_path)
        cells.x = cells.x * 2
        cells.y = cells.y *2
        
        # save
        new_dir = "data/ROI/region_csv_scale_corrected/LTX" * LTX * "PSR_20X.czi"
        new_path = new_dir * "/Da" * Da * ".csv"
        mkpath(new_dir)
        CSV.write(new_path, cells)
    end
end

# 3. Check the scale of deconvolved images and cell locations

In [17]:
# load all LTX_Da
LTX_Da = load("data/ROI/LTX_Da_dict.jld2")["LTX_Da"]
LTX_Da_green = load("data/ROI_ES_201222/deconvolved/LTX_Da_green_dict.jld2")["LTX_Da_green"]
LTX_Da_purple = load("data/ROI_ES_201222/deconvolved/LTX_Da_purple_dict.jld2")["LTX_Da_purple"];

Example image in `data/ROI`

In [7]:
LTX = "124"
Da = "104"

# load cell & img
cells = load_cells(LTX, Da)
img = load_ECM_image(LTX, Da)

# get leukocyte locations
leukocytes_loc = cells[cells.class .== "leukocytes", :];

In [ ]:
# check that leukocytes and ECM are aligned
gr()
p = plot(Gray.(img), frame = :box, ticks = [])
p1 = plot(Gray.(img), frame = :box, ticks = [])
p2 = scatter(leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
p3 = scatter!(p, leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
plot(p1, p2, p3, layout = grid(1,3), size = (800,200))

Example image in `data/ROI_ES_201222`, green

In [21]:
LTX = "021"
Da = "106"

# load cell & img
cells = load_cells(LTX, Da; green_purple = "green")
img = load_ECM_image(LTX, Da; green_purple = "green")

# get leukocyte locations
leukocytes_loc = cells[cells.class .== "leukocytes", :];

In [ ]:
# check that leukocytes and ECM are aligned
gr()
p = plot(Gray.(img), frame = :box, ticks = [])
p1 = plot(Gray.(img), frame = :box, ticks = [])
p2 = scatter(leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
p3 = scatter!(p, leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
plot(p1, p2, p3, layout = grid(1,3), size = (800,200))

Example image in `data/ROI_ES_201222`, purple

In [27]:
LTX = "021"
Da = "608"

# load cell & img
cells = load_cells(LTX, Da; green_purple = "purple")
img = load_ECM_image(LTX, Da; green_purple = "purple")

# get leukocyte locations
leukocytes_loc = cells[cells.class .== "leukocytes", :];

In [ ]:
# check that leukocytes and ECM are aligned
gr()
p = plot(Gray.(img), frame = :box, ticks = [])
p1 = plot(Gray.(img), frame = :box, ticks = [])
p2 = scatter(leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
p3 = scatter!(p, leukocytes_loc.x, leukocytes_loc.y, markersize= 2, label = "leukocytes", yflip = true, markerstrokewidth = 0.5)
plot(p1, p2, p3, layout = grid(1,3), size = (800,200))

# 4. Sample regions of size 4,000 x 4,000

Sample sub-regions from the images.
* Since we have a few images with width 4000 or height 4000, we'll sample sub-regions of size 4,000 by 4,000
* Let `m` denote the minimum of width and height
* If `m < 4000` we don't sample any
* If `m == 4000` sample one
* If `4000 < m <= 8000`, sample two
* If `8000 < m <= 12000`, sample three
* If `12000 < m `, sample four

* We require that the centers of subregions be at least 2000 pixels apart. 

In [2]:
# load all LTX_Da
LTX_Da = load("data/ROI/LTX_Da_dict.jld2")["LTX_Da"]
LTX_Da_green = load("data/ROI_ES_201222/deconvolved/LTX_Da_green_dict.jld2")["LTX_Da_green"]
LTX_Da_purple = load("data/ROI_ES_201222/deconvolved/LTX_Da_purple_dict.jld2")["LTX_Da_purple"];

## 4(a) for `data/ROI`

In [8]:
# get the size of all ROIs
ROI_size = Dict()
for (LTX, Da_list) in LTX_Da
    for Da in Da_list
        img = load_ECM_image(LTX, Da)
        ROI_size[(LTX, Da)] = size(img)
        println("size of image LTX ", LTX, " Da ", Da, ": ", size(img))
    end
end

#save("data/ROI/ROI_size.jld2", "ROI_size", ROI_size);

size of image LTX 091 Da 268: (12000, 8000)
size of image LTX 091 Da 355: (8000, 12000)
size of image LTX 109 Da 530: (8000, 12000)
size of image LTX 109 Da 621: (12000, 8000)
size of image LTX 156 Da 195: (8000, 16000)
size of image LTX 156 Da 353: (8000, 12000)
size of image LTX 221 Da 505: (12000, 16000)


┌ Info: Precompiling JpegTurbo [b835a17e-a41a-41e7-81f0-2f016b05efe0]
└ @ Base loading.jl:1342


size of image LTX 092 Da 104: (8000, 12000)
size of image LTX 092 Da 130: (8000, 12000)
size of image LTX 092 Da 252: (16000, 20000)
size of image LTX 092 Da 370: (8000, 16000)
size of image LTX 092 Da 378: (16000, 20000)
size of image LTX 092 Da 379: (12000, 16000)
size of image LTX 092 Da 440: (8000, 12000)
size of image LTX 092 Da 498: (8000, 8000)
size of image LTX 092 Da 523: (12000, 16000)
size of image LTX 092 Da 524: (8000, 8000)
size of image LTX 055 Da 409: (8000, 12000)
size of image LTX 055 Da 541: (12000, 16000)
size of image LTX 013 Da 148: (12000, 12000)
size of image LTX 013 Da 153: (12000, 12000)
size of image LTX 013 Da 154: (12000, 12000)
size of image LTX 013 Da 155: (12000, 12000)
size of image LTX 013 Da 168: (8000, 12000)
size of image LTX 013 Da 271: (12000, 12000)
size of image LTX 079 Da 105: (8000, 12000)
size of image LTX 079 Da 108: (8000, 8000)
size of image LTX 079 Da 50: (8000, 8000)
size of image LTX 142 Da 169: (8000, 8000)
size of image LTX 142 Da 201

In [68]:
subregion_centers = Dict()
subregion_size = 4000
distance_required = 3000

for (key, val) in ROI_size
    LTX, Da = key
    img_w, img_h = val
    if (img_w == 4000) | (img_h == 4000)
        if img_w == 4000
            sampled  = sample(Int(subregion_size//2): Int(img_h - subregion_size//2), 1)[1]
            sampled_loc = [[2000, sampled]]
        end
    
        if img_h == 4000
            sampled  = sample(Int(subregion_size//2): Int(img_w - subregion_size//2), 1)[1]
            sampled_loc = [[sampled, 2000]]
        end  
    else
        max_length = maximum([img_w, img_h])
        n_samples = floor.(Int,max_length / subregion_size)
        sampled_loc = sample_regions(img_w, img_h, subregion_size, distance_required; n_samples = n_samples)
    end
    subregion_centers[(LTX, Da)] = sampled_loc
end

In [70]:
print("total number of subregions: ", sum([length(v) for (_, v) in subregion_centers]))

total number of subregions: 268

In [ ]:
# plot example subregions
LTX = "124"
Da = "104"
plot_subregion_boxes(LTX, Da, subregion_centers[(LTX, Da)])

Save plots in `subregion_boxes`, `subregion_ECM`, and `sugregion_ECM_inverted`

In [2]:
# save subregion_centers
#save("data/4000x4000/subregion_centers.jld2", "subregion_centers", subregion_centers)

# to load
subregion_centers = load("data/4000x4000/subregion_centers.jld2")["subregion_centers"];

In [11]:
"""
# plot & save subregion center boxes
for (k, v) in subregion_centers
    LTX, Da = k
    plot_subregion_boxes(LTX, Da, subregion_centers[(LTX, Da)])
    savefig("data/4000x4000/subregion_boxes/LTX" * LTX * "_Da" * Da * ".png")
end
"""


In [402]:
# save subregions
"""
subregion_size = 4000
for (key, centers) in subregion_centers 
    if (key != ("143", "372"))
        (LTX, Da) = key

        # load ROI ECM 
        img = load_ECM_image(LTX, Da)
        for (idx,c) in enumerate(centers)
            # get subregion_ECM
            img_sub = get_img_sub(img, c, subregion_size)
            save("data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        end
    end
end
"""


In [ ]:
# save inverted version of small images 
"""
subregion_size = 4000
for (key, centers) in subregion_centers 
    if (key != ("143", "372"))
        (LTX, Da) = key
        # load ROI ECM 
        img = load_ECM_image(LTX, Da)
        for (idx,c) in enumerate(centers)
            # plot subregion ECM
            img_sub = get_img_sub(img, c, subregion_size)
            img_sub = 1 .- img_sub
            save("data/4000x4000/subregion_ECM_inverted/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        end
    end
end
"""

In [ ]:
# save the PSRH subregion images (with cell annotations)
subregion_size = 2000  # note: we halve the subegion size due to resolution issues
for (key, centers) in subregion_centers
    if (key != ("143", "372"))
        (LTX, Da) = key
        centers = subregion_centers[(LTX, Da)]
        img = Images.load("/media/yoon/My Passport/data/ROI/region_annotate_full/" *  "LTX" * LTX * "PSR_20X.czi/Da" * Da * ".jpg")
        
        for (idx,c) in enumerate(centers)
            # get subregion_ECM
            img_sub = get_img_sub(img, [Int(round(i/2)) for i in c] , subregion_size)
            save("data/4000x4000_combined/PSRH/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        end
    end
end

In [3]:
# save the PSRH subregion images 
subregion_size = 2000  # note: we halve the subregion size due to resolution issues
for (key, centers) in subregion_centers
    if (key != ("143", "372"))
        (LTX, Da) = key
        
        centers = subregion_centers[(LTX, Da)]
        img = Images.load("data/ROI/region_PSRH/" *  "LTX" * LTX * "PSR_20X.czi/Da" * Da * ".jpg")

        for (idx,c) in enumerate(centers)
            # get subregion_ECM
            img_sub = get_img_sub(img, [Int(round(i/2)) for i in c] , subregion_size)
            save("data/4000x4000_combined/PSRH/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        end

    end
end

<font color = "red">After gettin (unannotated) PSRH images from Crick:</color>
run the above again
LTX001 Da: 163, 256, 373
LTX013 Da: 148, 154, 271
LTX092 Da: 252, 378, 523

### save cell locations in subregions

In [399]:
for (key, val) in subregion_centers
    LTX, Da = key
    for (idx, c) in enumerate(val)
        center_x, center_y = c
        xmin, xmax, ymin, ymax = get_subregion_boundaries(center_x, center_y, subregion_size)

        cells = load_cells(LTX, Da)
        cells_sub = cells[(xmin .<= cells.y .< xmax) .& ( ymin .<= cells.x .< ymax),:]
        CSV.write("data/4000x4000/subregion_cells/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv", cells_sub)
    end
end

## 4(b) For new batch `ROI_ES_201222`

In [30]:
# get the size of all ROIs
ROI_green_size = Dict()
for (LTX, Da_list) in LTX_Da_green
    for Da in Da_list
        img = load_ECM_image(LTX, Da; green_purple = "green")
        ROI_green_size[(LTX, Da)] = size(img)
        println("size of image LTX ", LTX, " Da ", Da, ": ", size(img))
    end
end

ROI_purple_size = Dict()
for (LTX, Da_list) in LTX_Da_purple
    for Da in Da_list
        img = load_ECM_image(LTX, Da; green_purple = "purple")
        ROI_purple_size[(LTX, Da)] = size(img)
        println("size of image LTX ", LTX, " Da ", Da, ": ", size(img))
    end
end

save("data/ROI_ES_201222/deconvolved/ROI_size.jld2", 
    "ROI_green_size", ROI_green_size,
    "ROI_purple_size", ROI_purple_size);

size of image LTX 091 Da 295: (8000, 10000)
size of image LTX 091 Da 525: (8000, 8000)
size of image LTX 091 Da 896: (8000, 8000)
size of image LTX 109 Da 1225: (8000, 8000)
size of image LTX 109 Da 1569: (10000, 8000)
size of image LTX 156 Da 1195: (10000, 10000)
size of image LTX 156 Da 1596: (10000, 10000)
size of image LTX 156 Da 2042: (8000, 10000)
size of image LTX 156 Da 715: (10000, 10000)
size of image LTX 221 Da 1525: (10000, 10000)
size of image LTX 221 Da 2892: (10000, 10000)
size of image LTX 055 Da 1309: (10000, 12000)
size of image LTX 055 Da 2158: (12000, 12000)
size of image LTX 055 Da 738: (12000, 12000)
size of image LTX 142 Da 1112: (6000, 6000)
size of image LTX 142 Da 461: (6000, 8000)
size of image LTX 144 Da 1763: (6000, 6000)
size of image LTX 144 Da 2104: (8000, 8000)
size of image LTX 144 Da 554: (6000, 6000)
size of image LTX 210 Da 1477: (8000, 10000)
size of image LTX 210 Da 2682: (6000, 6000)
size of image LTX 160 Da 1980: (8000, 8000)
size of image LTX 1

#### Sample subregions for green

In [2]:
# sample subregion center for greens
ROI_size = load("data/ROI_ES_201222/deconvolved/ROI_size.jld2")["ROI_green_size"]
subregion_centers_green = sample_subregion_centers(ROI_size, 4000, 3000)

Dict{Any, Any} with 65 entries:
  ("051", "1230") => Any[[2882, 4674], [5989, 4692]]
  ("124", "955")  => Any[[3959, 2178]]
  ("193", "1320") => Any[[3905, 4689], [5823, 7400]]
  ("073", "1081") => Any[[3401, 2707]]
  ("156", "1596") => Any[[3615, 7043], [5383, 4332]]
  ("185", "733")  => Any[[5749, 3769], [6883, 8933], [3340, 9813]]
  ("100", "1257") => Any[[3383, 2492], [4326, 6900], [8795, 3260]]
  ("049", "1488") => Any[[6705, 4725], [2461, 6619]]
  ("050", "1764") => Any[[3920, 3359], [2155, 5852]]
  ("050", "2137") => Any[[5056, 4853], [4845, 7937]]
  ("108", "1561") => [[2000, 2024]]
  ("206", "2168") => Any[[3391, 9972], [7700, 7284], [5078, 2254]]
  ("091", "525")  => Any[[4969, 4275], [2298, 2008]]
  ("221", "2892") => Any[[2944, 4214], [6015, 7198]]
  ("021", "385")  => Any[[4141, 3128], [5577, 5955]]
  ("108", "808")  => Any[[2980, 2131], [3958, 5881]]
  ("100", "2090") => Any[[5472, 5484], [2159, 3088]]
  ("082", "920")  => Any[[3968, 6038], [5809, 2991]]
  ("183", "1489")

In [5]:
print("total number of subregions: ", sum([length(v) for (_, v) in subregion_centers_green]))

total number of subregions: 128

In [3]:
# save subregion_centers
#save("data/4000x4000_201222/subregion_centers_green.jld2", "subregion_centers_green", subregion_centers_green)

# to load
subregion_centers_green = load("data/4000x4000_201222/subregion_centers_green.jld2")["subregion_centers_green"];

In [ ]:

# plot & save subregion center boxes
"""
for (k, v) in subregion_centers_green
    LTX, Da = k
    plot_subregion_boxes(LTX, Da, subregion_centers_green[(LTX, Da)]; green_purple = "green")
    savefig("data/4000x4000_201222/subregion_boxes_green/LTX" * LTX * "_Da" * Da * ".png")
end
"""


In [11]:
# save subregions
subregion_size = 4000
for (key, centers) in subregion_centers_green

    (LTX, Da) = key

    # load ROI ECM 
    img = load_ECM_image(LTX, Da; green_purple = "green")
    for (idx,c) in enumerate(centers)
        # get subregion_ECM
        img_sub = get_img_sub(img, c, subregion_size)
        #save("data/4000x4000_201222/subregion_ECM_green/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        
        # to save inverted version
        img_sub = Float32.(1 .- img_sub)
        save("data/4000x4000_combined/subregion_ECM_inverted_green_purple/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
    end
end



In [6]:
# save the PSRH subregion images 
subregion_size = 4000 
    (LTX, Da) = key
    img = Images.load("data/ES_201222/stitch_region_40x_green/region_cws/201222/LTX" * LTX * "PSR_20X.czi/Da" * Da * ".jpg")
    for (idx,c) in enumerate(centers)
        # get subregion_ECM
        img_sub = get_img_sub(img, c , subregion_size)
        save("data/4000x4000_201222/subregion_PSRH_green/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
    end

end

In [4]:
# save cell locations
subregion_size = 4000
for (key, val) in subregion_centers_green
    LTX, Da = key
    for (idx, c) in enumerate(val)
        center_x, center_y = c
        xmin, xmax, ymin, ymax = get_subregion_boundaries(center_x, center_y, subregion_size)

        cells = load_cells(LTX, Da; green_purple = "green")
        cells_sub = cells[(xmin .<= cells.y .< xmax) .& ( ymin .<= cells.x .< ymax),:]
        CSV.write("data/4000x4000_combined/subregion_cells/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv", cells_sub)
    end
end

#### Sample subregions for purple

In [12]:
# sample subregion center for greens
ROI_size = load("data/ROI_ES_201222/deconvolved/ROI_size.jld2")["ROI_purple_size"]
subregion_centers_purple = sample_subregion_centers(ROI_size, 4000, 3000)

Dict{Any, Any} with 7 entries:
  ("100", "913")  => [[2000, 2000]]
  ("210", "1832") => [[2000, 2000]]
  ("021", "608")  => [[2097, 2000]]
  ("051", "1513") => [[2000, 2000]]
  ("049", "1564") => [[2000, 3974]]
  ("050", "1433") => [[3097, 2000]]
  ("097", "721")  => [[2000, 2000]]

In [13]:
print("total number of subregions: ", sum([length(v) for (_, v) in subregion_centers_purple]))

total number of subregions: 7

In [14]:
# save subregion_centers
#save("data/4000x4000_201222/subregion_centers_purple.jld2", "subregion_centers_purple", subregion_centers_purple)

# to load
subregion_centers_purple = load("data/4000x4000_201222/subregion_centers_purple.jld2")["subregion_centers_purple"];

In [ ]:

# plot & save subregion center boxes
"""
for (k, v) in subregion_centers_purple
    LTX, Da = k
    plot_subregion_boxes(LTX, Da, subregion_centers_purple[(LTX, Da)]; green_purple = "purple")
    savefig("data/4000x4000_201222/subregion_boxes_purple/LTX" * LTX * "_Da" * Da * ".png")
end
"""


In [15]:
# save subregions
subregion_size = 4000
for (key, centers) in subregion_centers_purple

    (LTX, Da) = key

    # load ROI ECM 
    img = load_ECM_image(LTX, Da; green_purple = "purple")
    for (idx,c) in enumerate(centers)
        # get subregion_ECM
        img_sub = get_img_sub(img, c, subregion_size)
        #save("data/4000x4000_201222/subregion_ECM_purple/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
        
        # to save inverted version
        img_sub = Float32.(1 .- img_sub)
        save("data/4000x4000_combined/subregion_ECM_inverted_green_purple/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
    end
end



In [11]:
# save the PSRH subregion images 
subregion_size = 4000 
for (key, centers) in subregion_centers_purple

    (LTX, Da) = key
    img = Images.load("data/ES_201222/stitch_region_40x_purple/region_cws/LTX" * LTX * "PSR_20X.czi/Da" * Da * ".jpg")
    for (idx,c) in enumerate(centers)
        # get subregion_ECM
        img_sub = get_img_sub(img, c , subregion_size)
        save("data/4000x4000_201222/subregion_PSRH_purple/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif", img_sub)
    end
end

In [6]:
# save cell locations
subregion_size = 4000
for (key, val) in subregion_centers_purple
    LTX, Da = key
    for (idx, c) in enumerate(val)
        center_x, center_y = c
        xmin, xmax, ymin, ymax = get_subregion_boundaries(center_x, center_y, subregion_size)

        cells = load_cells(LTX, Da; green_purple = "purple")
        cells_sub = cells[(xmin .<= cells.y .< xmax) .& ( ymin .<= cells.x .< ymax),:]
        CSV.write("data/4000x4000_combined/subregion_cells/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv", cells_sub)
    end
end